In [1]:
import pyspark as ps

## Experimenting with PySpark

In [2]:
sc = ps.SparkContext()

In [3]:
link = 's3n://mortar-example-data/airline-data'

http://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236

In [4]:
airline = sc.textFile(link)

In [5]:
airline.take(2)

[u'"YEAR","MONTH","UNIQUE_CARRIER","ORIGIN_AIRPORT_ID","DEST_AIRPORT_ID","DEP_DELAY","DEP_DELAY_NEW","ARR_DELAY","ARR_DELAY_NEW","CANCELLED",',
 u'2012,4,"AA",12478,12892,-4.00,0.00,-21.00,0.00,0.00,']

In [6]:
airline_no_quote = airline.map(lambda line: line.replace('\'', '').replace('\"', '').strip(','))

In [7]:
import time

start = time.time()

airline_no_quote.take(2)

end = time.time()

print end - start

14.1967048645


In [8]:
airline_no_quote.cache()

PythonRDD[4] at RDD at PythonRDD.scala:43

In [9]:
start = time.time()

results = airline_no_quote.take(2)

end = time.time()

print "Time Elapsed: {0} seconds".format(end - start)

Time Elapsed: 14.0089240074 seconds


In [10]:
start = time.time()

results = airline_no_quote.take(2)

end = time.time()

print "Time Elapsed: {0} seconds".format(end - start)

Time Elapsed: 0.0287280082703 seconds


In [11]:
results

[u'YEAR,MONTH,UNIQUE_CARRIER,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,DEP_DELAY,DEP_DELAY_NEW,ARR_DELAY,ARR_DELAY_NEW,CANCELLED',
 u'2012,4,AA,12478,12892,-4.00,0.00,-21.00,0.00,0.00']

In [12]:
header_line = airline_no_quote.first()

header_list = header_line.split(',')

print header_list

[u'YEAR', u'MONTH', u'UNIQUE_CARRIER', u'ORIGIN_AIRPORT_ID', u'DEST_AIRPORT_ID', u'DEP_DELAY', u'DEP_DELAY_NEW', u'ARR_DELAY', u'ARR_DELAY_NEW', u'CANCELLED']


In [13]:
airline_no_header = airline_no_quote.filter(lambda row: row != header_list)

In [14]:
airline_no_header.first()

u'YEAR,MONTH,UNIQUE_CARRIER,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,DEP_DELAY,DEP_DELAY_NEW,ARR_DELAY,ARR_DELAY_NEW,CANCELLED'

In [15]:
airline_no_header = airline_no_quote.filter(lambda row: row != header_line)
airline_no_header.first()

u'2012,4,AA,12478,12892,-4.00,0.00,-21.00,0.00,0.00'

In [16]:
def make_row(row):
    row_list = row.split(',')
    
    d = dict(zip(header_list, row_list))
    
    return d

In [17]:
airline_rows = airline_no_header.map(make_row)

In [18]:
airline_rows.take(5)

[{u'ARR_DELAY': u'-21.00',
  u'ARR_DELAY_NEW': u'0.00',
  u'CANCELLED': u'0.00',
  u'DEP_DELAY': u'-4.00',
  u'DEP_DELAY_NEW': u'0.00',
  u'DEST_AIRPORT_ID': u'12892',
  u'MONTH': u'4',
  u'ORIGIN_AIRPORT_ID': u'12478',
  u'UNIQUE_CARRIER': u'AA',
  u'YEAR': u'2012'},
 {u'ARR_DELAY': u'-65.00',
  u'ARR_DELAY_NEW': u'0.00',
  u'CANCELLED': u'0.00',
  u'DEP_DELAY': u'-7.00',
  u'DEP_DELAY_NEW': u'0.00',
  u'DEST_AIRPORT_ID': u'12892',
  u'MONTH': u'4',
  u'ORIGIN_AIRPORT_ID': u'12478',
  u'UNIQUE_CARRIER': u'AA',
  u'YEAR': u'2012'},
 {u'ARR_DELAY': u'-63.00',
  u'ARR_DELAY_NEW': u'0.00',
  u'CANCELLED': u'0.00',
  u'DEP_DELAY': u'-6.00',
  u'DEP_DELAY_NEW': u'0.00',
  u'DEST_AIRPORT_ID': u'12892',
  u'MONTH': u'4',
  u'ORIGIN_AIRPORT_ID': u'12478',
  u'UNIQUE_CARRIER': u'AA',
  u'YEAR': u'2012'},
 {u'ARR_DELAY': u'5.00',
  u'ARR_DELAY_NEW': u'5.00',
  u'CANCELLED': u'0.00',
  u'DEP_DELAY': u'-6.00',
  u'DEP_DELAY_NEW': u'0.00',
  u'DEST_AIRPORT_ID': u'12892',
  u'MONTH': u'4',
  u'ORIGI

In [19]:
destination_rdd = airline_rows.map(lambda row: (row['DEST_AIRPORT_ID'], \
                                                float(row['ARR_DELAY'] if row['ARR_DELAY'] else 0)))
origin_rdd = airline_rows.map(lambda row: (row['ORIGIN_AIRPORT_ID'], \
                                           float(row['DEP_DELAY']) if row['DEP_DELAY'] else 0))

In [20]:
destination_rdd.take(2)

[(u'12892', -21.0), (u'12892', -65.0)]

In [21]:
origin_rdd.take(2)

[(u'12478', -4.0), (u'12478', -7.0)]

## Mean Delay for Take off and landing

In [22]:
import numpy as np

mean_delays_dest = destination_rdd.groupByKey().mapValues(lambda delays: np.mean(delays.data))
mean_delays_origin = origin_rdd.groupByKey().mapValues(lambda delays: np.mean(delays.data))

In [23]:
mean_delays_origin.take(1)

[(u'10141', -1.8319672131147542)]

In [24]:
print mean_delays_origin.sortBy(lambda t: t[1], ascending=True).take(10)

[(u'12129', -6.7547169811320753), (u'15991', -6.0978441127694856), (u'12888', -5.9056603773584904), (u'14113', -5.3462002412545235), (u'10779', -5.1457627118644069), (u'13127', -5.0891265597147948), (u'14633', -4.9087677725118484), (u'10739', -4.666666666666667), (u'15897', -4.6107142857142858), (u'11274', -4.6034482758620694)]


In [25]:
print mean_delays_origin.sortBy(lambda t: t[1], ascending=False).take(10)

[(u'13541', 33.845454545454544), (u'10930', 32.533490011750878), (u'13964', 30.258196721311474), (u'10157', 24.69469773725093), (u'15295', 20.405063291139239), (u'15356', 20.399999999999999), (u'14487', 20.05645784996133), (u'12016', 19.472491909385113), (u'13424', 18.739763421292082), (u'11002', 17.139170506912443)]


In [26]:
print mean_delays_dest.sortBy(lambda t: t[1], ascending=True).take(10)

[(u'12343', -16.925233644859812), (u'12888', -8.9444444444444446), (u'11415', -7.5650793650793648), (u'10466', -7.4375), (u'12335', -5.6448598130841123), (u'10158', -5.5479452054794525), (u'13127', -5.1532976827094474), (u'10141', -4.8770491803278686), (u'10779', -4.7915254237288138), (u'11695', -4.472236911686938)]


In [27]:
print mean_delays_dest.sortBy(lambda t: t[1], ascending=False).take(10)

[(u'10930', 24.724705882352943), (u'13424', 20.856232939035486), (u'14487', 20.795507358636716), (u'13459', 20.609195402298852), (u'10157', 18.995945945945945), (u'12177', 15.940357852882704), (u'12016', 15.879084967320262), (u'13964', 15.779753761969904), (u'11002', 15.714811407543698), (u'14771', 12.516759209337824)]
